In [ ]:
using Pkg
using JuMP
using Gurobi
#using GLPK
using XLSX
using DataFrames

In [ ]:
using LinearAlgebra

In [ ]:
shared_cand = DataFrame(XLSX.readtable("../Input_Data/CandLine.csv", "Taul1", header=true)...) #Dataframe of shared candidate lines
int_cand =  DataFrame(XLSX.readtable("../Input_Data/CandLineInt.csv", "Taul1", header=true)...) #Dataframe of internal candidate lines
shared_ex =  DataFrame(XLSX.readtable("../Input_Data/SharedEline.csv", "Taul1", header=true)...) #Dataframe of shared existing lines
int_ex = DataFrame(XLSX.readtable("../Input_Data/Tran.csv", "Taul1", header=true)...) #Dataframe of internal existing lines
gen =  DataFrame(XLSX.readtable("../Input_Data/Gen.csv" , "Taul1", header=true)...) #Dataframe of generators
load =  DataFrame(XLSX.readtable("../Input_Data/Load.csv", "Taul1", header=true)...) #Dataframe of loads
##An example of the load dataframe can be found as follows

In [ ]:
cand_from(i) = (shared_cand.nodeZone1 .== i)
cand_to(i) = (shared_cand.nodeZone2 .== i)
ex_from(i) = (shared_ex.nodeZone1 .== i)
ex_to(i) = (shared_ex.nodeZone2 .== i)

In [ ]:
l(i) = load[load.zoneNum .== i, :] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1 .* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i

In [ ]:
m(i) = setdiff([1,2,3], [i])

In [ ]:
function zonal_app(i, N, cand_lambda, ex_lambda,shared_line_decision_var, int_line_decision_var,
        shared_cand_angle_self, shared_ex_angle_self, shared_cand_angle_1, shared_ex_angle_1, 
        shared_cand_angle_2, shared_ex_angle_2)
    
    Mod4 = Model(Gurobi.Optimizer)
    
    
    @variable(Mod4,0 <= gen_var[1:N]) # generation at each node
    @variable(Mod4,shared_cand_flow[1:nrow(shared_cand)])  #Power flowing into shared candidate lines 
    @variable(Mod4,int_cand_flow[1:nrow(int_c(i))]) #Power flowing into internal candidate lines 
    @variable(Mod4,0 <= shared_cand_angle[1:nrow(shared_cand),1:2]<= 2*pi) #Phase angle decision for shared candidate lines
    @variable(Mod4,0 <= int_cand_angle[1:nrow(int_c(i)),1:2]<= 2*pi) #Phase angle decision for internal candidate lines
    @variable(Mod4,shared_ex_flow[1:nrow(shared_ex)])  #Power flowing on shared existing shared lines
    @variable(Mod4,int_ex_flow[1:nrow(int_e(i))])  #Power flowing on internal existing lines
    @variable(Mod4,0 <= shared_ex_angle[1:nrow(shared_ex),1:2]<= 2*pi) #Phase angle for existing shared lines
    @variable(Mod4,0 <= int_ex_angle[1:nrow(int_e(i)),1:2]<= 2*pi) #Phase angle for internal existing lines
    
    omega = 0.16;
    eta = 0.4;
    
     # Each zone i considered to have shared lines with two other zones, we call it other zone 1 and other zone 2:
     #shared_ex_angle_self[h,1]: from angle of existing line h within zone i:
     #shared_ex_angle_self[h,2]: to angle of existing line h within zone i:
     #shared_ex_angle_1[h,1]: from angle of existing line h within the other zone 1:
     #shared_ex_angle_2[h,2]: to angle of existing line h within the other zone 2:
    @expression(Mod4, total_cost, sum((gen_var[n] .* sum((g(i).gNodeID .== n) .* MC(i))) for n in 1:N)
        .+ sum(sum(eta * bin_e(i) .* shared_ex_angle[h,1] .* (shared_ex_angle_self[h,1] .- shared_ex_angle_1[h,2]) .+
            eta * bin_e(i) .* shared_ex_angle[h,1] .* (shared_ex_angle_self[h,1] .- shared_ex_angle_2[h,2]) .+
            eta * bin_e(i) .* shared_ex_angle[h,2] .* (shared_ex_angle_self[h,2] .- shared_ex_angle_1[h,1]) .+
            eta * bin_e(i) .* shared_ex_angle[h,2] .* (shared_ex_angle_self[h,2] .- shared_ex_angle_2[h,1]) .+
            bin_e(i) .* shared_ex_angle[h,1] .* ex_lambda[h,1] .+ 
            bin_e(i) .* shared_ex_angle[h,2] .* ex_lambda[h,2] for h in 1:nrow(shared_ex))) .+
            sum(sum((omega /2) * bin_e(i) .* (shared_ex_angle[:,1] .- shared_ex_angle_self[:,1]) .^ 2)) .+
            sum(sum((omega /2) * bin_e(i) .* (shared_ex_angle[:,2] .- shared_ex_angle_self[:,2]) .^ 2))
        .+ sum(sum(eta * bin_c(i) .* shared_cand_angle[c,1] .* (shared_cand_angle_self[c,1] .- shared_cand_angle_1[c,2]) .+
            eta * bin_c(i) .* shared_cand_angle[c,1] .* (shared_cand_angle_self[c,1] .- shared_cand_angle_2[c,2]) .+
            eta * bin_c(i) .* shared_cand_angle[c,2] .* (shared_cand_angle_self[c,2] .- shared_cand_angle_1[c,1]) .+
            eta * bin_c(i) .* shared_cand_angle[c,2] .* (shared_cand_angle_self[c,2] .- shared_cand_angle_2[c,1]) .+
            bin_c(i) .* shared_cand_angle[c,1] .* cand_lambda[c,1] .+ 
            bin_c(i) .* shared_cand_angle[c,2] .* cand_lambda[c,2] for c in 1:nrow(shared_cand))) .+
            sum(sum((omega /2) * bin_c(i) .* (shared_cand_angle[:,1] .- shared_cand_angle_self[:,1]) .^ 2)) .+
            sum(sum((omega /2) * bin_c(i) .* (shared_cand_angle[:,2] .- shared_cand_angle_self[:,2]) .^ 2)))
            
            
            
            
    for n in 1:N
        # Power balance constraint for each node
        @constraint(Mod4, sum(g(i).gNodeID .== n) .* gen_var[n] .+ sum(l(i).P_load1 .* (l(i).lNodeID .== n)) .==
            sum((shared_cand.tNodeID1 .== n) .*bin_c(i) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(i) .* shared_cand_flow) .+
            sum((shared_ex.tNodeID1 .== n) .* bin_e(i) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(i) .* shared_ex_flow) .+
            sum((int_c(i).tNodeID1 .== n) .* int_cand_flow) .- sum((int_c(i).tNodeID2 .== n) .* int_cand_flow) .+
            sum((int_e(i).tNodeID1 .== n) .* int_ex_flow) .- sum((int_e(i).tNodeID2 .== n) .* int_ex_flow))
        #Lower limit for generation of each node
        @constraint(Mod4, sum(g(i).gNodeID .== n) .* gen_var[n] .<= sum((g(i).gNodeID .== n) .* g(i).PgMax))
        #Upper limit for generation of each node
        @constraint(Mod4, sum((g(i).gNodeID .== n) .* g(i).PgMin) .<= sum(g(i).gNodeID .== n) .* gen_var[n])
    end
    
    M=1000 #Big number
    
    #Constraints elated to the candidate shared lines:
    for c in 1:nrow(shared_cand) 
        #Associating the power flowing within the line with the phase angles of nodes:
        @constraint(Mod4,-M .* (1 .- bin_c(i) .* shared_line_decision_var[c]) .<= bin_c(i) .* shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (bin_c(i) .* shared_cand_angle[c,1] .- bin_c(i) .* shared_cand_angle[c,2])))
        
        @constraint(Mod4, bin_c(i) .* shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (bin_c(i) .* shared_cand_angle[c,1] .- bin_c(i) .* shared_cand_angle[c,2])) .<= M .* (1 .- bin_c(i) .* shared_line_decision_var[c]))
        #limiting the upper bound of power flow flowing within candidate shared lines
        @constraint(Mod4, bin_c(i) .* shared_cand_flow[c] .<= bin_c(i) .* shared_line_decision_var[c] .* shared_cand.ptMax[c])
        #Limiting the lower bound of power flowing within the candidate shared lines
        @constraint(Mod4, -shared_cand.ptMax[c] .* bin_c(i) .* shared_line_decision_var[c] .<= bin_c(i) .* shared_cand_flow[c])
        @constraint(Mod4, shared_cand_angle[5,2] .== 0)
    end
    
    #The same constraints related to the candidate internal lines:
    for c in 1:nrow(int_c(i))
        @constraint(Mod4,-M .* (1 .- int_line_decision_var[c]) .<= int_cand_flow[c] .- ((1 ./ int_c(i).reacT[c]) .* (int_cand_angle[c,1] .- int_cand_angle[c,2])))
        @constraint(Mod4, int_cand_flow[c] .- ((1 ./ int_c(i).reacT[c]) .* (int_cand_angle[c,1] .- int_cand_angle[c,2])) .<= M .* (1 .- int_line_decision_var[c]))
        @constraint(Mod4, int_cand_flow[c] .<= int_line_decision_var[c] .* int_c(i).ptMax[c])
        @constraint(Mod4, -int_c(i).ptMax[c] .*int_line_decision_var[c] .<= int_cand_flow[c])
    end
    
    #The same constraints related to the shared existing lines:
    for h in 1:nrow(shared_ex)
        @constraint(Mod4, bin_e(i) .* shared_ex_flow[h] .== (1 ./ shared_ex.reacT[h]) .* (bin_e(i) .* shared_ex_angle[h,1] .- bin_e(i) .* shared_ex_angle[h,2]))
        @constraint(Mod4, bin_e(i) .* shared_ex_flow[h] .<= bin_e(i) .* shared_ex. ptMax[h])
        @constraint(Mod4, bin_e(i) .* -shared_ex.ptMax[h] .<= bin_e(i) .* shared_ex_flow[h])
        @constraint(Mod4, shared_ex_angle[1,2] .== 0)
        @constraint(Mod4, shared_ex_angle[7,2] .== 0)

    end
    #The same constraints related to the internal existing lines:
    
    for h in 1:nrow(int_e(i))
        @constraint(Mod4, int_ex_flow[h] .== (1 ./ int_e(i).reacT[h]) .* (int_ex_angle[h,1] .- int_ex_angle[h,2]))
        @constraint(Mod4, int_ex_flow[h] .<= int_e(i).ptMax[h])
        @constraint(Mod4, -int_e(i).ptMax[h] .<= int_ex_flow[h])
    end
    
    #The model objective is to minimize the total cost:
    
    @objective(Mod4, Min, total_cost)
    optimize!(Mod4)
    if termination_status(Mod4) == MOI.OPTIMAL #If the solution is optimal then it returns the variables
    
        shared_cand_power = value.(shared_cand_flow)
        shared_cand_phase_angle = value.(shared_cand_angle)
        shared_ex_power = value.(shared_ex_flow)
        shared_ex_phase_angle = value.(shared_ex_angle)
        gen_power = value.(gen_var)
        int_cand_power = value.(int_cand_flow)
        int_cand_phase_angle = value.(int_cand_angle)
        obj_value = objective_value(Mod4)
        [obj_value, shared_cand_power,shared_cand_phase_angle,shared_ex_power,shared_ex_phase_angle,gen_power, int_cand_power, int_cand_phase_angle]

    end
end
    
    

In [ ]:
f=ones(12000)
cand_lambda_1= zeros(12000,nrow(shared_cand),2)
ex_lambda_1 = zeros(12000,nrow(shared_ex),2)
cand_lambda_2= zeros(12000,nrow(shared_cand),2)
ex_lambda_2 = zeros(12000,nrow(shared_ex),2)
cand_lambda_3= zeros(12000,nrow(shared_cand),2)
ex_lambda_3 = zeros(12000,nrow(shared_ex),2)
cand_angle_1 = zeros(12000,nrow(shared_cand),2)
c_angle_1 = zeros(12000,nrow(shared_cand),2)
cand_angle_2 = zeros(12000,nrow(shared_cand),2)
c_angle_2 = zeros(12000,nrow(shared_cand),2)
cand_angle_3 = zeros(12000,nrow(shared_cand),2)
c_angle_3 = zeros(12000,nrow(shared_cand),2)

ex_angle_1 = zeros(12000,nrow(shared_ex),2)
e_angle_1 = zeros(12000,nrow(shared_ex),2)
ex_angle_2 = zeros(12000,nrow(shared_ex),2)
e_angle_2 = zeros(12000,nrow(shared_ex),2)
ex_angle_3 = zeros(12000,nrow(shared_ex),2)
e_angle_3 = zeros(12000,nrow(shared_ex),2)




shared_line_decision_var_m =  
[0.0
 0.0
 0.0
 1.0
 1.0
 1.0]
int_line_decision_var_1 =
[ 0.0
 1.0
 1.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 1.0]
int_line_decision_var_2 =
[1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0]
int_line_decision_var_3 =
[0.0
 0.0]

In [ ]:
v6=LinRange(0.1, 10, 50);
obj = zeros(length(v6));
h=1;

In [ ]:
for k in 1:length(v6)
    for j in 2:100    # It considers 1000 iteration
        if abs(f[j]) > 0.00001  
            alpha = 0.25
            t1=1
            t2=1
            t3=1
            beta1 = 0.5
            beta2 = 0.5
            beta3 = 0.5
            revised_objective = 100
            original_objective = objective_fn(cand_lambda, ex_lambda)
            if(revised_objective>=original_objective+alpha*t*sum(gradient_obj().*deltax))
                original_objective = objective_fn(cand_lambda, ex_lambda)
                R_1 = zonal_app(1, 14, cand_lambda_1[j,:,:], ex_lambda_1[j,:,:],shared_line_decision_var_m, int_line_decision_var_1,
                cand_angle_1[j-1,:,:], ex_angle_1[j-1,:,:], cand_angle_2[j-1,:,:],  ex_angle_2[j-1,:,:], 
                cand_angle_3[j-1,:,:], ex_angle_3[j-1,:,:])
        
                R_2 = zonal_app(2, 30, cand_lambda_2[j,:,:], ex_lambda_2[j,:,:],shared_line_decision_var_m, int_line_decision_var_2,
                cand_angle_2[j-1,:,:], ex_angle_2[j-1,:,:], cand_angle_1[j-1,:,:], ex_angle_1[j-1,:,:], 
                cand_angle_3[j-1,:,:], ex_angle_3[j-1,:,:])       
        
                R_3 = zonal_app(3, 5, cand_lambda_3[j,:,:], ex_lambda_3[j,:,:],shared_line_decision_var_m, int_line_decision_var_3,
                cand_angle_3[j-1,:,:], ex_angle_3[j-1,:,:], cand_angle_1[j-1,:,:],  ex_angle_1[j-1,:,:], 
                cand_angle_2[j-1,:,:], ex_angle_2[j-1,:,:])
        
        
                cand_lambda_1[j+1,:,:] .= cand_lambda_1[j,:,:] .+ t1 * (R_1[3] .- (R_2[3] .+ R_3[3]) ./2)
                cand_lambda_2[j+1,:,:] .= cand_lambda_2[j,:,:] .+ t2 * (R_2[3] .- (R_1[3] .+ R_3[3]) ./2)
                cand_lambda_3[j+1,:,:] .= cand_lambda_3[j,:,:] .+ t3 * (R_3[3] .- (R_1[3] .+ R_2[3]) ./2)
        
        
    
                ex_lambda_1[j+1,:,:] .= ex_lambda_1[j,:,:] .+ t1 * (R_1[5] .- (R_2[5] .+ R_3[5]) ./2)
                ex_lambda_2[j+1,:,:] .= ex_lambda_2[j,:,:] .+ t2 * (R_2[5] .- (R_1[5] .+ R_3[5]) ./2)
                ex_lambda_3[j+1,:,:] .= ex_lambda_3[j,:,:] .+ t3 * (R_3[5] .- (R_1[5] .+ R_2[5]) ./2)

                t1 = beta1*t1
                t2 = beta2*t2
                t3 = beta3*t3

                revised_objective = objective_fn(cand_lambda, ex_lambda)
        
                f[j] = sum(sum((R_1[3] .- R_2[3] .- R_3[3]) .^ 2)) + sum(sum((R_2[3] .- R_1[3] .- R_3[3]) .^ 2)) + sum(sum((R_3[3] .- R_1[3] .- R_2[3]) .^2)) +sum(sum((R_1[5] .- R_2[5] .- R_3[5]) .^ 2)) + sum(sum((R_2[5] .- R_1[5] .- R_3[5]) .^ 2)) + sum(sum((R_3[5] .- R_1[5] .- R_2[5]) .^2))
        
                c_angle_1 = R_1[3]
                c_angle_2 = R_2[3]
                c_angle_3 = R_3[3]
        
                e_angle_1 = R_1[5]
                e_angle_2 = R_2[5]
                e_angle_3 = R_3[5]
            
                h= R_3[1] + R_2[1] +R_1[1]
            
                if isdefined(Main, :IJulia)
                    Main.IJulia.stdio_bytes[] = 0
                end

            end
        
            print('*') 
        
        else
            print(c_angle_1)
            print(c_angle_2)
            print(c_angle_3)
            print(e_angle_1)
            print(e_angle_2)
            print(e_angle_3)
        end
    end
   obj[k]=h;
    
end

function objective_fn(cand_lambda, ex_lambda)
    @expression(Mod4, total_cost, sum((gen_var[n] .* sum((g(i).gNodeID .== n) .* MC(i))) for n in 1:N)
    .+ sum(sum(eta * bin_e(i) .* shared_ex_angle[h,1] .* (shared_ex_angle_self[h,1] .- shared_ex_angle_1[h,2]) .+
        eta * bin_e(i) .* shared_ex_angle[h,1] .* (shared_ex_angle_self[h,1] .- shared_ex_angle_2[h,2]) .+
        eta * bin_e(i) .* shared_ex_angle[h,2] .* (shared_ex_angle_self[h,2] .- shared_ex_angle_1[h,1]) .+
        eta * bin_e(i) .* shared_ex_angle[h,2] .* (shared_ex_angle_self[h,2] .- shared_ex_angle_2[h,1]) .+
        bin_e(i) .* shared_ex_angle[h,1] .* ex_lambda[h,1] .+ 
        bin_e(i) .* shared_ex_angle[h,2] .* ex_lambda[h,2] for h in 1:nrow(shared_ex))) .+
        sum(sum((omega /2) * bin_e(i) .* (shared_ex_angle[:,1] .- shared_ex_angle_self[:,1]) .^ 2)) .+
        sum(sum((omega /2) * bin_e(i) .* (shared_ex_angle[:,2] .- shared_ex_angle_self[:,2]) .^ 2))
    .+ sum(sum(eta * bin_c(i) .* shared_cand_angle[c,1] .* (shared_cand_angle_self[c,1] .- shared_cand_angle_1[c,2]) .+
        eta * bin_c(i) .* shared_cand_angle[c,1] .* (shared_cand_angle_self[c,1] .- shared_cand_angle_2[c,2]) .+
        eta * bin_c(i) .* shared_cand_angle[c,2] .* (shared_cand_angle_self[c,2] .- shared_cand_angle_1[c,1]) .+
        eta * bin_c(i) .* shared_cand_angle[c,2] .* (shared_cand_angle_self[c,2] .- shared_cand_angle_2[c,1]) .+
        bin_c(i) .* shared_cand_angle[c,1] .* cand_lambda[c,1] .+ 
        bin_c(i) .* shared_cand_angle[c,2] .* cand_lambda[c,2] for c in 1:nrow(shared_cand))) .+
        sum(sum((omega /2) * bin_c(i) .* (shared_cand_angle[:,1] .- shared_cand_angle_self[:,1]) .^ 2)) .+
        sum(sum((omega /2) * bin_c(i) .* (shared_cand_angle[:,2] .- shared_cand_angle_self[:,2]) .^ 2)))
end

function gradient_obj()

end

In [ ]:
using Plots
plot(v6,obj,title = "Objective function in terms of ex_lambda_3 par")
xlabel!("ex_lambda_3 parameter")
ylabel!("Objective function value")

In [ ]:
h = findall((0 .<= obj) .* (obj .<= 70));
v6[h]

In [ ]:
c_angle_1

In [ ]:
c_angle_2

In [ ]:
c_angle_3

In [ ]:
e_angle_3 

In [ ]:
e_angle_2

In [ ]:
e_angle_1